In [1]:
import pyperclip
from tf.app import use
A = use('etcbc/bhsa')
F = A.TF.api.F
L = A.TF.api.L

def get_lexeme_nodes(book, chapter=None, alphabetical=False):
    ch_spec = "chapter=%d"%(chapter) if chapter else ""
    # Find lexeme-word pairs
    tuples = A.search('''
l:lex
    w:word
    /with/
    book book=%s
        chapter %s
            w
    /-/
    '''%(book, ch_spec))

    if alphabetical:
        # Sort by word feature `lex_utf8`
        tuples.sort(key=lambda t: F.lex_utf8.v(t[1]))
    else:
        # Sort by word nodes (which are in canonical order)
        tuples.sort(key=lambda t: t[1])

    # Select lexeme nodes only and remove duplicates
    lexemes = list({ l: w for (l, w) in tuples })

    print("%d lexemes found in %s %s"%(len(lexemes), book, str(chapter or "")))
    return lexemes

def table_lexemes(list, limit=None):
    limit = limit or 100000
    A.table([(l,) for l in list][0:limit])

def copy_zipped_lists(lists, vertical=True):
    # separate items with newline and tab
    sep_col = "\t"
    sep_row = "\n" 

    # construct string
    if vertical:
        rows = zip(*lists)
    else:
        rows = lists

    string = ""
    for r in rows:
        string += sep_col.join(str(i) for i in r) + sep_row

    # remove trailing separator
    if string[-1:] == sep_row:
        string = string[:-1]

    pyperclip.copy(string)

def copy_lexemes(lexemes, features=["voc_lex_utf8"], vertical=True):
    data = []
    for f_name in features:
        # see Text-Fabric Api for details
        # e.g., getter for 'freq_lex' would be `F.freq_lex.v`
        val_getter = getattr(F, f_name).v

        val_list = [val_getter(i) for i in lexemes]
        data.append(val_list)
    
    copy_zipped_lists(data, vertical)

This is Text-Fabric 9.3.2
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

122 features found and 0 ignored


In [2]:
# Jeremiah 23 (with vowels, without vowels, frequency)
lexemes = get_lexeme_nodes("Jeremia", 23)
copy_lexemes(lexemes, ["voc_lex_utf8", "lex_utf8", "freq_lex"])

  0.15s 801 results
215 lexemes found in Jeremia 23
